# SmartBeds - One-Class - Pruebas con otros datos
## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import pickle as pk
from sklearn import metrics as mtr
from transformers import *

### Carga

In [2]:
with open('data/crisis_separadas.pkl','rb') as f:
    crisis_separadas = pk.load(f)

In [3]:
segunda = crisis_separadas[2]
segunda.head()

,DateTime,P1,P2,P3,P4,P5,P6,target
2164084,2019-02-06 21:11:50,0.0,14.754098,27.049180,30.327869,10.655738,0.0,False
2164085,2019-02-06 21:11:51,0.0,15.573770,26.229508,24.590164,6.557377,0.0,False
2164087,2019-02-06 21:11:52,0.0,19.672131,29.508197,23.770492,4.918033,0.0,False
2164086,2019-02-06 21:11:52,0.0,18.032787,30.327869,27.049180,7.377049,0.0,False
2164088,2019-02-06 21:11:53,0.0,18.032787,25.409836,17.213115,0.000000,0.0,False


In [4]:
segunda_datos = segunda.iloc[:,1:len(segunda.columns)-1]
segunda_datos.head()

,P1,P2,P3,P4,P5,P6
2164084,0.0,14.754098,27.049180,30.327869,10.655738,0.0
2164085,0.0,15.573770,26.229508,24.590164,6.557377,0.0
2164087,0.0,19.672131,29.508197,23.770492,4.918033,0.0
2164086,0.0,18.032787,30.327869,27.049180,7.377049,0.0
2164088,0.0,18.032787,25.409836,17.213115,0.000000,0.0


### Transformers

In [5]:
svg = SavgolTransformer(15)
btr = ButterTransformer(3,0.05)
nor = Normalizer(max_=100)
avg = PipelineTransformer(StatisticsTransformer(mode='mean',window=25),nor)
std = PipelineTransformer(StatisticsTransformer(mode='std',window=25),nor)
ran = PipelineTransformer(StatisticsTransformer(mode='range',window=25),nor)
var = VarianceThresholdPD(threshold=0.5)
con = ConcatenateTransformer(avg,std,ran)
#Se quita el ajuste de varianza para mantener el mismo número de entradas
pps = PipelineTransformer(svg,con)
ppb = PipelineTransformer(btr,con)
ppr = PipelineTransformer(con)
ppr_s = PipelineTransformer(svg)
ppr_b = PipelineTransformer(btr)

### Brutos

In [6]:
with open('classifiers/RAW-seizure-classifier.pkl','rb') as f:
    raw_clf = pk.load(f)

In [7]:
predict = raw_clf.predict(segunda_datos)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

array([False, False, False, ..., False, False, False])

In [8]:
mtr.confusion_matrix(segunda['target'],predict)

array([[86990,     0],
       [ 2741,     0]])

### Estadísticos

In [9]:
with open('classifiers/STATS-seizure-classifier.pkl','rb') as f:
    sts_clf = pk.load(f)

In [10]:
prd = con.fit_transform(segunda_datos).dropna()
predict = sts_clf.predict(prd)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

array([False, False, False, ..., False, False, False])

In [11]:
target = (pd.concat((prd,segunda['target']),axis=1)).dropna()['target']
target.head()

2164108    False
2164109    False
2164110    False
2164111    False
2164112    False
Name: target, dtype: bool

In [12]:
mtr.confusion_matrix(target,predict)

array([[86966,     0],
       [ 2741,     0]])

### SavGol

In [13]:
with open('classifiers/SAVGOL-seizure-classifier.pkl','rb') as f:
    sav_clf = pk.load(f)

In [15]:
sav = ppr_s.fit_transform(segunda_datos).dropna()
predict = sav_clf.predict(sav)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

array([False, False, False, ..., False, False, False])

In [16]:
target = (pd.concat((sav,segunda['target']),axis=1)).dropna()['target']
target.head()

2164084    False
2164085    False
2164087    False
2164086    False
2164088    False
Name: target, dtype: bool

In [17]:
mtr.confusion_matrix(target,predict)

array([[86990,     0],
       [ 2741,     0]])

### Butter

In [18]:
with open('classifiers/BUTTER-seizure-classifier.pkl','rb') as f:
    btt_clf = pk.load(f)

In [19]:
but = ppr_b.fit_transform(segunda_datos).dropna()
predict = btt_clf.predict(but)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


array([False, False, False, ..., False, False, False])

In [20]:
target = (pd.concat((but,segunda['target']),axis=1)).dropna()['target']
target.head()

2164084    False
2164085    False
2164087    False
2164086    False
2164088    False
Name: target, dtype: bool

In [21]:
mtr.confusion_matrix(target,predict)

array([[86990,     0],
       [ 2741,     0]])

### SavGol Estadísticos

In [22]:
with open('classifiers/SAVGOL-STATS-seizure-classifier.pkl','rb') as f:
    sst_clf = pk.load(f)

In [23]:
sst = pps.fit_transform(segunda_datos).dropna()
predict = sts_clf.predict(sst)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


array([False, False, False, ..., False, False, False])

In [24]:
target = (pd.concat((sst,segunda['target']),axis=1)).dropna()['target']
target.head()

2164108    False
2164109    False
2164110    False
2164111    False
2164112    False
Name: target, dtype: bool

In [25]:
mtr.confusion_matrix(target,predict)

array([[86966,     0],
       [ 2741,     0]])

### Butter Estadísticos

In [26]:
with open('classifiers/BUTTER_STATS-seizure-classifier.pkl','rb') as f:
    bts_clf = pk.load(f)

In [27]:
bts = ppb.fit_transform(segunda_datos).dropna()
predict = sts_clf.predict(bts)
predict[predict==-1]=0
predict = predict.astype('bool')
predict

/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


array([False, False, False, ..., False, False, False])

In [28]:
target = (pd.concat((bts,segunda['target']),axis=1)).dropna()['target']
target.head()

2164108    False
2164109    False
2164110    False
2164111    False
2164112    False
Name: target, dtype: bool

In [29]:
mtr.confusion_matrix(target,predict)

array([[86966,     0],
       [ 2741,     0]])